## 1. Import or generate datasets

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

data = tfds.load("iris", split="train")

2022-01-17 17:02:46.965407: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/150 [00:00<?, ? examples/s]

Shuffling iris-train.tfrecord...:   0%|          | 0/150 [00:00<?, ? examples/s]

Dataset iris downloaded and prepared to /home/wil/tensorflow_datasets/iris/2.0.0. Subsequent calls will reuse this data.


2022-01-17 17:02:48.176550: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-01-17 17:02:48.250768: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 17:02:48.251133: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-17 17:02:48.251147: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-01-17 17:02:48.260132: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-01-17 17:02:48.260162: I tensorflow/stream_executor/plat

## 2. Transform and normalize data

In [17]:
# Note that while the book describes this process, step 4 is required to define batch_size first.
for batch in data.batch(batch_size, drop_remainder=True):
    labels = tf.one_hot(batch['label'], 3)
    X = batch['features']
    X = (X - np.mean(X)) / np.std(X)

## 3. Partition the dataset into training, test, and validation sets

We generally want to test our algorithms on different sets that we have trained on. Many algorithms also require hyperparameter tuning, so we set aside a validation set for determining the best set of hyperparameters.

## 4. Set algorithm parameters (hyperparameters)

In [14]:
epochs = 1000
batch_size = 32
input_size = 4
output_size = 3
learning_rate = 0.001

## 5. Initialize variables

In [15]:
weights = tf.Variable(tf.random.normal(shape=(input_size, output_size), dtype=tf.float32))
biases = tf.Variable(tf.random.normal(shape=(output_size,), dtype=tf.float32))

## 6. Define the model structure

In [16]:
logits = tf.add(tf.matmul(X, weights), biases)

2022-01-17 17:14:23.988010: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-01-17 17:14:24.470760: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


## 7. Declare the loss functions

In [18]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, logits))

## 8. Initialize and train the model

In [21]:
optimizer = tf.optimizers.SGD(learning_rate)

with tf.GradientTape() as tape:
    logits = tf.add(tf.matmul(X, weights), biases)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels, logits))
gradients = tape.gradient(loss, [weights, biases])
optimizer.apply_gradients(zip(gradients, [weights, biases]))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

## 9. Evaluate the model

In [24]:
print(f"final loss is {loss.numpy():.3f}")
preds = tf.math.argmax(tf.add(tf.matmul(X, weights), biases), axis=1)
ground_truth = tf.math.argmax(labels, axis=1)
for y_true, y_pred in zip(ground_truth.numpy(), preds.numpy()):
    print(f"real label: {y_true}, fitted: {y_pred}")

final loss is 1.521
real label: 0, fitted: 2
real label: 1, fitted: 2
real label: 1, fitted: 2
real label: 2, fitted: 2
real label: 2, fitted: 2
real label: 2, fitted: 2
real label: 0, fitted: 1
real label: 2, fitted: 2
real label: 1, fitted: 2
real label: 2, fitted: 2
real label: 1, fitted: 2
real label: 0, fitted: 1
real label: 1, fitted: 2
real label: 0, fitted: 1
real label: 2, fitted: 2
real label: 2, fitted: 2
real label: 0, fitted: 2
real label: 2, fitted: 2
real label: 0, fitted: 2
real label: 1, fitted: 2
real label: 2, fitted: 2
real label: 0, fitted: 2
real label: 2, fitted: 2
real label: 1, fitted: 2
real label: 0, fitted: 1
real label: 0, fitted: 1
real label: 2, fitted: 2
real label: 0, fitted: 1
real label: 1, fitted: 2
real label: 2, fitted: 2
real label: 0, fitted: 2
real label: 2, fitted: 2


## 10. Tune Hyperparameters

Most of the time, we will want to go back and change some of the hyperparameters, checking the models performance based on our tests. We then repeat the previous steps with different hyperparameters and evaluate the model on the validation set.

## 11. Deploy/predict new outcomes

It is also a key requirement to know how to make predictions on new and unseen data. We can achieve this easily with TensorFlow with all of our models once we have them trained.